In [1]:
# === SETUP ===
import pulp
import os
from pprint import pprint

# Portable solver setup, to allow work locally as well as in JupyterHub
from pulp import COIN_CMD
if os.path.exists("/opt/homebrew/opt/cbc/bin/cbc"):
    solver = COIN_CMD(path="/opt/homebrew/opt/cbc/bin/cbc", msg=0)
else:
    solver = pulp.PULP_CBC_CMD(msg=0)

In [2]:
# Define our three drivers and customers as lists
drivers = [0, 1, 2] 
customers = [0, 1, 2] 

print(f"Our drivers are {drivers}")
print(f"Our customers are {customers}")

Our drivers are [0, 1, 2]
Our customers are [0, 1, 2]


In [3]:
# Input our data on travel times as a dictionary
travel_times = {
    (0,0): 1, # minutes
    (0,1): 4,
    (0,2): 6,
    (1,0): 3,
    (1,1): 1,
    (1,2): 6,
    (2,0): 8,
    (2,1): 8,
    (2,2): 12
}
print("These are travel times between each driver and customer:")
pprint(travel_times)

# e.g., Driver 2 and Customer 1 is 8 minutes apart
example_driver = drivers[2]
example_customer = customers[1]
example_pair = (example_driver, example_customer)
example_distance = travel_times[example_pair]
print(f"e.g., Driver {example_driver} and Customer {example_customer} is {example_distance} minutes apart")

These are travel times between each driver and customer:
{(0, 0): 1,
 (0, 1): 4,
 (0, 2): 6,
 (1, 0): 3,
 (1, 1): 1,
 (1, 2): 6,
 (2, 0): 8,
 (2, 1): 8,
 (2, 2): 12}
e.g., Driver 2 and Customer 1 is 8 minutes apart


In [4]:
# Initialize the optimization model
model = pulp.LpProblem("Uber", pulp.LpMinimize)

In [5]:
# Define all 9 decision variables as X_Di_Cj, where
# Di is the ith Driver
# Cj is the jth Customer
X = {}
for i in drivers:
    for j in customers:
        variable_name = f"X_D{i}_C{j}"
        X[(i,j)] = pulp.LpVariable(variable_name, cat="Binary")
pprint(X)

{(0, 0): X_D0_C0,
 (0, 1): X_D0_C1,
 (0, 2): X_D0_C2,
 (1, 0): X_D1_C0,
 (1, 1): X_D1_C1,
 (1, 2): X_D1_C2,
 (2, 0): X_D2_C0,
 (2, 1): X_D2_C1,
 (2, 2): X_D2_C2}


In [11]:
# Objective: minimize sum of travel times between matched drivers and customers
obj = None
for i in drivers:
    for j in customers:
        obj += travel_times[(i,j)] * X[(i,j)]
model += obj

In [7]:
# Add driver constraints: one driver must be matched to one customer
for i in drivers:
    const = None
    for j in customers:
        const += X[(i,j)]
    model += const == 1, f"driver_{i}"

In [8]:
# Add customer constraints: one customer must be matched to one driver
for j in customers:
    const = None
    for i in drivers:
        const += X[(i,j)]
    model += const == 1, f"customer_{j}"

In [9]:
# Run solver
model.solve(solver)
print(f"Status: {pulp.LpStatus[model.status]}\n")

# Print report (pairs matched and total travel time)
print(model)
print(f"Total travel time = {pulp.value(model.objective)} minutes")
for v in model.variables():
    print(f"{v.name} = {v.varValue}")

Status: Optimal

Uber:
MINIMIZE
1*X_D0_C0 + 4*X_D0_C1 + 6*X_D0_C2 + 3*X_D1_C0 + 1*X_D1_C1 + 6*X_D1_C2 + 8*X_D2_C0 + 8*X_D2_C1 + 12*X_D2_C2 + 0
SUBJECT TO
driver_0: X_D0_C0 + X_D0_C1 + X_D0_C2 = 1

driver_1: X_D1_C0 + X_D1_C1 + X_D1_C2 = 1

driver_2: X_D2_C0 + X_D2_C1 + X_D2_C2 = 1

customer_0: X_D0_C0 + X_D1_C0 + X_D2_C0 = 1

customer_1: X_D0_C1 + X_D1_C1 + X_D2_C1 = 1

customer_2: X_D0_C2 + X_D1_C2 + X_D2_C2 = 1

VARIABLES
0 <= X_D0_C0 <= 1 Integer
0 <= X_D0_C1 <= 1 Integer
0 <= X_D0_C2 <= 1 Integer
0 <= X_D1_C0 <= 1 Integer
0 <= X_D1_C1 <= 1 Integer
0 <= X_D1_C2 <= 1 Integer
0 <= X_D2_C0 <= 1 Integer
0 <= X_D2_C1 <= 1 Integer
0 <= X_D2_C2 <= 1 Integer

Total travel time = 14.0 minutes
X_D0_C0 = 1.0
X_D0_C1 = 0.0
X_D0_C2 = 0.0
X_D1_C0 = 0.0
X_D1_C1 = 1.0
X_D1_C2 = 0.0
X_D2_C0 = 0.0
X_D2_C1 = 0.0
X_D2_C2 = 1.0
